In [1]:
# Duty cycle calculation

Vd = 0.5 # Diode voltage drop [V]
Vo = 8.4 # Output voltage [V]
Vinmin = 3.3 # Minimum input voltage [V]
Vinnom = 3.7 # Nominal input voltage [V]
Vinmax = 4.2 # Maximum input voltage [V]

Dmin = (Vo + Vd - Vinmax)/(Vo + Vd)
Dnom = (Vo + Vd - Vinnom)/(Vo + Vd)
Dmax = (Vo + Vd - Vinmin)/(Vo + Vd)

# Topics
* Maximum stress at Vinmin, where duty cycle is maximum, so highest peak current.
* Disadvantages of boost converter:
    * Input over-voltage can't be stopped
    * Output short circuit can't be stopped

# References
* [Boost Converter: Design, Circuit, Equations & More, part 1: Jurgen Hubner](https://www.powerelectronicsnews.com/the-dc-dc-boost-converter-power-supply-design-tutorial-section-5-1/)
* [Boost Converter: Design, Circuit, Equations & More, part 2: Jurgen Hubner](https://www.powerelectronicsnews.com/the-dc-dc-boost-converter-part-2-power-supply-design-tutorial-section-5-2/)

In [14]:
# Two ways for average inductor current calculation
D = Dmax
Io = 2 # Output current [A]
ηest = 0.8 # Estimated efficiency
Vin = Vinmin

# Method 1
Il1 = Io / ((1-D)*ηest)

# Method 2
Il2 = Io * Vo / (Vin * ηest)

Iinmax = max(Il1, Il2)

In [15]:
# Inductor selection
ΔIlpp = Il * 0.3 # Peak-to-peak inductor ripple current [A] -> design choice
fsw = 600e3 # Switching frequency [Hz]
Ilmin = 0.5 # Minimum inductor current [A]

Lmin1 = Dmax * Vinmin / (ΔIlpp * fsw)

# Check for discontinuous mode (DCM)
Lmin2 = Dmin * Vinmax / (2*Ilmin * fsw)

L  = 2.2e-6 # Inductor value [H] -> chosen value

# Threshold current between CCM and DCM
Ith = Dmin * Vinmax / (2*L*fsw)

# Calculate actual peak-to-peak inductor ripple current
ΔIlpp = Dmax * Vinmin / (L*fsw)

# Calculate actual peak inductor current
Ilpk = Iinmax + ΔIlpp/2

# Inductor part selection

1. Inductor saturation current must be higher than the maximum peak current (Ilpk) of the inductor.
2. There's no standard for inductor RMS current.  Select one that has an acceptable temperature rise at the maximum peak current.

# Input capacitor
1. For simulation, use 200nH in series with a 15mOhm resistor to simulate an imperfect voltage source.  Otherwise the input capacitors won't do anything.
2. Take care of DC-derating of capacitors.
3. To avoid oscillation, output impedance of input filter must be lower than input impedance of the converter.
4. Input filter damping for MLCC capacitors:
    * Add Cin2 = 4*Cin1 parallel to Cin1
    * provide room for an extra damping resistor in series with Cin2

In [21]:
import math
# Input impedance of the converter
Iomax = 2 # Maximum output current [A]
Zin = - Vinmin**2 / (Vo * Iomax)

Rs = 0.1 # Resistance of the input leads [Ohm]
ESR_Cin1 = 10e-3 # Equivalent series resistance of the input capacitor [Ohm]
ESR_Cin2 = 0.1 # Equivalent series resistance of the input capacitor [Ohm]
Rd_in = 0 # any discreet resistor placed in series with the damping capacitor Cin2
Lin = 1e-6 # input inductance.  If unknown, set to 1µH
Cin1 = 20e-6 # input capacitance, MLCC
Cin2 = 100e-6 # damping capacitance, elco (or polymer for high temperature)

Zs = math.sqrt(Lin/(Cin1+Cin2))

# Calculate the damping factor
δin = (Rs + ESR_Cin1 + ESR_Cin2 + Rd_in) / (2*Zs) + Zs / (2 * abs(Zin))

Damping factor δin should be > 1.

In [ ]:
# Input capacitor RMS current

# Total RMS current
Icin_rms = ΔIlpp / math.sqrt(12)

# Calculate the reactance of each capacitor (using 8 instead of 2 * pi because wave is triangular, not sinusoidal)
Xc1 = 1 / (8 * fsw * Cin1)
Zc1 = math.sqrt(ESR_Cin1**2 + Xc1**2)

Xc2 = 1 / (8 * fsw * Cin2)
Zc2 = math.sqrt(ESR_Cin2**2 + Xc2**2)

# Calculate the RMS current in each capacitor
Ic1_rms = Icin_rms * Zc2 / (Zc1 + Zc2)
Ic2_rms = Icin_rms * Zc1 / (Zc1 + Zc2)

# Increase MLCC to avoid elco from overheating

In [28]:
# Defining output diode currents

# Average diode current = average inductor current = average input current
Id_avg = Iomax / (1-Dmax) # Average diode current [A]

# Peak diode current = peak inductor current
Id_pk = Ilpk

Id_rms = math.sqrt((1-Dmax) * (Id_pk**2 + ΔIlpp**2/3 - Id_pk * ΔIlpp))
Ico_rms = math.sqrt(Id_rms**2 - Iomax**2)

Cout1 = 20e-6 # output capacitance, MLCC
Cout2 = 100e-6 # output capacitance, elco (or polymer for high temperature)
ESR_Cout1 = 10e-3 # Equivalent series resistance of the output capacitor [Ohm]
ESR_Cout2 = 0.1 # Equivalent series resistance of the output capacitor [Ohm]

# Output capacitor calculation
Xco1 = 1 / (8 * fsw * Cout1)
Zco1 = math.sqrt(ESR_Cout1**2 + Xco1**2)

Xco2 = 1 / (8 * fsw * Cout2)
Zco2 = math.sqrt(ESR_Cout2**2 + Xco2**2)

Ico1_rms = Ico_rms * Zco2 / (Zco1 + Zco2)
Ico2_rms = Ico_rms * Zco1 / (Zco1 + Zco2)

# Increase MLCC to avoid elco from overheating

In [30]:
# Output voltage ripple
Δvomaxpp = Vo * 0.03 # Maximum output voltage ripple [V] -> design choice
Iopp = 1 # Output current ripple, due to load changes [A] -> design choice

Co_min1 = 2 * Io * Dmax / (Δvomaxpp * fsw)
ESR_max1 = Δvomaxpp / ( 2 * Id_pk)

Δvopp = ΔIlpp + Iopp / (1 / Zco1  + 1 / Zco2)

In [33]:
# Load transient response
# Calculate the worst case right-hand pole zero 

ΔIload = 1 # Load step [A] -> design choice
Δvo_load = 0.1 # Load step [V] -> design choice

f_rhpz_min = (Vo / Iomax) * (Vinmin/Vo)**2 / (2 * math.pi * L)

Co_min2 = 2 * ΔIload / (math.pi * f_rhpz_min * Δvo_load)
ESR_max2 = Δvo_load / (2 * ΔIload)

Co_min = max(Co_min1, Co_min2)
ESR_max = min(ESR_max1, ESR_max2)

In [ ]:
# Output diode selection
Vr = Vo * 1.2 # Reverse voltage [V] -> design choice
Idmax = Iomax * 1.2 # Maximum diode current [A] -> design choice

Diode package selection : PowerDI5 for lower inductance.

# Second stage output filter design
[DESIGNING SECOND STAGE OUTPUT FILTERS FOR SWITCHING POWER SUPPLIES, Kevin M. Tompsett](https://www.analog.com/media/en/technical-documentation/tech-articles/Designing-Second-Stage-Output-Filters-for-Switching-Power-Supplies.pdf)

[The DC-DC Boost Converter, Part 3 – Power Supply Design Tutorial Section 5-3](https://www.powerelectronicsnews.com/the-dc-dc-boost-converter-part-3-power-supply-design-tutorial-section-5-3/)